# Explore here

In [10]:
import pandas as pd

In [12]:
# Your code here
trips_data = pd.read_csv("../data/raw/csv/combined_csv.csv", sep=",")

C:\Users\Rubén\AppData\Local\Temp\ipykernel_3812\2450552943.py:2: DtypeWarning: Columns (0,1,2,6,7,8,9,10,11,12,13,14,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  trips_data = pd.read_csv("../data/raw/csv/combined_csv.csv", sep=",")


# Clean up dataset

In [15]:
trips_data.drop("fecha,idTrip,idBike,fleet,trip_minutes,geolocation_unlock,address_unlock,unlock_date,locktype,unlocktype,geolocation_lock,address_lock,lock_date,station_unlock,dock_unlock,unlock_station_name,station_lock,dock_lock,lock_station_name", axis=1, inplace=True)

In [16]:
trips_data.dropna(how="all", inplace=True)

In [17]:
trips_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4144134 entries, 4144135 to 12432401
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   fecha                object 
 1   idTrip               object 
 2   idBike               float64
 3   fleet                float64
 4   trip_minutes         float64
 5   geolocation_unlock   object 
 6   address_unlock       object 
 7   unlock_date          object 
 8   locktype             object 
 9   unlocktype           object 
 10  geolocation_lock     object 
 11  address_lock         object 
 12  lock_date            object 
 13  station_unlock       object 
 14  dock_unlock          float64
 15  unlock_station_name  object 
 16  station_lock         float64
 17  dock_lock            float64
 18  lock_station_name    object 
dtypes: float64(6), object(13)
memory usage: 632.3+ MB


In [21]:
# Convierte la columna 'fecha' a tipo de dato datetime si aún no lo está
trips_data['fecha'] = pd.to_datetime(trips_data['fecha'])

# Ordena el DataFrame por la columna 'fecha'
trips_data = trips_data.sort_values(by='fecha')

# Muestra las primeras filas del DataFrame ordenado
trips_data.head()

,fecha,idTrip,idBike,fleet,trip_minutes,geolocation_unlock,address_unlock,unlock_date,locktype,unlocktype,geolocation_lock,address_lock,lock_date,station_unlock,dock_unlock,unlock_station_name,station_lock,dock_lock,lock_station_name
4144135,2022-01-01,489978239_1718_2022-01-01T00:18:37,1718.0,1.0,16.28,"{'type': 'Point', 'coordinates': [-3.6714166, ...",NaN,2022-01-01T00:02:20,STATION,STATION,"{'type': 'Point', 'coordinates': [-3.688398, 4...",NaN,2022-01-01T00:18:37,200.0,3.0,Avenida de los Toreros,64.0,4.0,Plaza de la Independencia
4144137,2022-01-01,06023769T_7340_2022-01-01T00:14:59,7340.0,1.0,7.10,"{'type': 'Point', 'coordinates': [-3.6894193, ...",NaN,2022-01-01T00:07:53,STATION,STATION,"{'type': 'Point', 'coordinates': [-3.6993465, ...",NaN,2022-01-01T00:14:59,69.0,5.0,Antonio Maura,169.0,17.0,Manuel Silvela
4144139,2022-01-01,50856526F_3861_2022-01-01T00:09:50,3861.0,1.0,0.48,"{'type': 'Point', 'coordinates': [-3.688822, 4...",NaN,2022-01-01T00:09:21,STATION,STATION,"{'type': 'Point', 'coordinates': [-3.688822, 4...",NaN,2022-01-01T00:09:50,73.0,21.0,Puerta del Ángel Caído,73.0,21.0,Puerta del Ángel Caído
4144141,2022-01-01,79006741Q_7657_2022-01-01T00:16:29,7657.0,1.0,6.62,"{'type': 'Point', 'coordinates': [-3.6653055, ...",NaN,2022-01-01T00:09:52,STATION,STATION,"{'type': 'Point', 'coordinates': [-3.6657777, ...",NaN,2022-01-01T00:16:29,192.0,22.0,Marqués de Zafra,190.0,17.0,Parque Roma
4144143,2022-01-01,76959007H_6653_2022-01-01T00:18:01,6653.0,1.0,8.07,"{'type': 'Point', 'coordinates': [-3.6983055, ...",NaN,2022-01-01T00:09:57,STATION,STATION,"{'type': 'Point', 'coordinates': [-3.7025024, ...",NaN,2022-01-01T00:18:01,183.0,3.0,Jaime el Conquistador,47.0,24.0,Jesús y María


In [42]:
trips_data["fleet"].value_counts()



1.0    4111082
2.0      33052
Name: fleet, dtype: int64

In [36]:
null_counts = trips_data.isnull().sum()
print(null_counts)


fecha                        0
idTrip                 2041145
idBike                       0
fleet                        0
trip_minutes                 0
geolocation_unlock           0
address_unlock          548474
unlock_date                  0
locktype                     0
unlocktype                   0
geolocation_lock             0
address_lock            547602
lock_date                    0
station_unlock           16792
dock_unlock              16792
unlock_station_name      16792
station_lock             19907
dock_lock                19907
lock_station_name        19907
dtype: int64


revisar nulos (idTrip)

Despues de arreglar nulos en idTrip, splitear columna segun codigo de abajo:

In [ ]:
'''# Separar la columna "idTrip" en varias columnas
trips_data[['DNI', 'idBike2', 'Fecha2', 'Hora_Fin']] = trips_data['idTrip'].str.split('_|-', expand=True)

# Convertir la columna 'Fecha' a tipo de dato datetime
trips_data['Fecha2'] = pd.to_datetime(trips_data['Fecha2'])

# Mostrar el DataFrame con las nuevas columnas
print(trips_data)'''
